In [1]:
import random
import pandas as pd

# Các chủ đề
topics = ["smartphone", "clothes", "laptops", "shoes", "books"]
# Tạo danh sách người dùng
users = [f"user_{i}" for i in range(1, 501)]

# Tạo dữ liệu comment
comments_data = []
for user in users:
    for _ in range(50):  # Mỗi user 50 comment
        comment = f"This is a comment about {random.choice(topics)}"
        topic = random.choice(topics)
        timestamp = pd.Timestamp(
            year=random.randint(2020, 2023),
            month=random.randint(1, 12),
            day=random.randint(1, 28),
            hour=random.randint(0, 23),
            minute=random.randint(0, 59),
            second=random.randint(0, 59),
        )
        comments_data.append({"User": user, "Comment": comment, "Topic": topic, "Timestamp": timestamp})

# Chuyển thành DataFrame
df_comments = pd.DataFrame(comments_data)

# Lưu dữ liệu vào file CSV
file_path = "user_comments.csv"
df_comments.to_csv(file_path, index=False)

print(f"Data saved to {file_path}")


Data saved to user_comments.csv


In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk

# Tải dữ liệu
df = pd.read_csv("user_comments.csv")

# Tải stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Tiền xử lý bình luận
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # Loại bỏ ký tự đặc biệt
    words = word_tokenize(text)
    return [word for word in words if word not in stop_words]

df["Processed_Comment"] = df["Comment"].apply(preprocess_text)

# Tạo Word2Vec model
comments_corpus = df["Processed_Comment"].tolist()
w2v_model = Word2Vec(comments_corpus, vector_size=100, window=5, min_count=1, workers=4)

# Tính vector trung bình cho từng bình luận
def comment_vector(comment, model):
    vectors = [model.wv[word] for word in comment if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df["Comment_Vector"] = df["Processed_Comment"].apply(lambda x: comment_vector(x, w2v_model))

# Xác định vector tích cực và tiêu cực
positive_words = ["good", "excellent", "like", "love", "amazing"]
negative_words = ["bad", "poor", "dislike", "terrible", "awful"]

positive_vector = np.mean([w2v_model.wv[word] for word in positive_words if word in w2v_model.wv], axis=0)
negative_vector = np.mean([w2v_model.wv[word] for word in negative_words if word in w2v_model.wv], axis=0)

# Phân loại cảm xúc
def classify_sentiment(comment_vec):
    if np.all(comment_vec == 0):  # Nếu vector rỗng
        return "Neutral"
    pos_score = cosine_similarity([comment_vec], [positive_vector])[0][0]
    neg_score = cosine_similarity([comment_vec], [negative_vector])[0][0]
    if pos_score > neg_score:
        return "Like"
    elif neg_score > pos_score:
        return "Dislike"
    else:
        return "Neutral"

df["Sentiment"] = df["Comment_Vector"].apply(classify_sentiment)

# Tóm tắt kết quả
sentiment_summary = df.groupby("User")["Sentiment"].apply(lambda x: x.value_counts().idxmax())
df["User_Sentiment"] = df["User"].map(sentiment_summary)

# Lưu dữ liệu phân loại ra file CSV
output_path = "user_sentiment_analysis.csv"
df.to_csv(output_path, index=False)

import ace_tools as tools; tools.display_dataframe_to_user(name="Sentiment Analysis Results", dataframe=df)

output_path


ImportError: cannot import name 'triu' from 'scipy.linalg' (c:\Users\ADMIN\anaconda3\Lib\site-packages\scipy\linalg\__init__.py)